In [ ]:
import io
import base64
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from loguru import logger
import numpy as np
import time
import pickle
import h5py
import umap
import scipy
import time

import itk
from itkwidgets import view
import SimpleITK as sitk

from utilities import plot_all_slices_notzero

import resampling

In [ ]:
nifti_img = nib.load("processing/134/shoulder_nifti.nii")
sx, sy, sz = nifti_img.header.get_zooms()
sx, sy, sz

In [ ]:
print(nifti_img.affine)

In [ ]:
x_spacing = []
y_spacing = []
z_spacing = []
for i in range(1,1251):
    if os.path.exists(f"processing/{i}/shoulder_nifti.nii"):
        nifti_img = nib.load(f"processing/{i}/shoulder_nifti.nii")
        sx, sy, sz = nifti_img.header.get_zooms()
        x_spacing.append(sx)
        y_spacing.append(sy)
        z_spacing.append(sz)
    else:
        x_spacing.append([])
        y_spacing.append([])
        z_spacing.append([]) 

In [ ]:
x_spacing_hom = []
y_spacing_hom = []
z_spacing_hom = []
for i in range(1,1251):
    if os.path.exists(f"processing/{i}/shoulder_nifti.nii"):
        nifti_img = nib.load(f"processing/{i}/shoulder_nifti.nii")
        sx, sy, sz = nifti_img.header.get_zooms()
        x_spacing_hom.append(sx)
        y_spacing_hom.append(sy)
        z_spacing_hom.append(sz)
x_spacing_hom = np.asarray(x_spacing_hom)
y_spacing_hom = np.asarray(y_spacing_hom)
z_spacing_hom = np.asarray(z_spacing_hom)

In [ ]:
np.median(x_spacing_hom),np.median(y_spacing_hom),np.median(z_spacing_hom)

In [ ]:
np.mean(x_spacing_hom),np.mean(y_spacing_hom),np.mean(z_spacing_hom)

In [ ]:
# Escludo CT 1173 dai calcoli
x_spacing[1172] = []
y_spacing[1172] = []
z_spacing[1172] = []
x_spacing_hom = np.delete(x_spacing_hom,1137)
y_spacing_hom = np.delete(y_spacing_hom,1137) # Indice della CT 1173
z_spacing_hom = np.delete(z_spacing_hom,1137)

In [ ]:
np.mean(x_spacing_hom),np.mean(y_spacing_hom),np.mean(z_spacing_hom)

In [ ]:
np.median(x_spacing_hom),np.median(y_spacing_hom),np.median(z_spacing_hom)

In [ ]:
min(x_spacing_hom),max(x_spacing_hom),min(y_spacing_hom),max(y_spacing_hom),min(z_spacing_hom),np.max(z_spacing_hom)

In [ ]:
z_spacing.index(min(z_spacing))

In [ ]:
y_spacing.index(max(y_spacing))

# Calcolo CT da escludere

Tengo quelle con spacing in x e y tra 0.35 e 0.65, in z tra 0.3 e 1 e quelle non inclinate

In [ ]:
# In x prendo da 0.35 a 0.65, anche y tanto è uguale
excluded = []
for i,x in enumerate(x_spacing,start=1):
    if not isinstance(x,list):
        if x < 0.35 or x > 0.65:
            excluded.append(i)
            print(i,x)

excluded_y = []
for i,x in enumerate(y_spacing,start=1):
    if not isinstance(x,list):
        if x < 0.35 or x > 0.65:
            excluded_y.append(i)
            print(i,x)

In [ ]:
len(excluded)

In [ ]:
# z da 0.3 a 1
for i,x in enumerate(z_spacing,start=1):
    if not isinstance(x,list):
        if x < 0.3 or x > 1:
            if i not in excluded: # Aggiungo ad excluded quelle da escludere in z che non ci sono già
                excluded.append(i)
            print(i,x)

# 0.3-1 in z
# 0.5 +- 0.15 in x e y

In [ ]:
# Aggiungo ad excluded_x quelle di excluded_z non presenti
len(excluded)

In [ ]:
# Escludo anche quelle con rotazione
rotated_cts = []
for i in range(1,1251):
    if os.path.exists(f"processing/{i}/shoulder_nifti.nii"):
        nifti_img = nib.load(f"processing/{i}/shoulder_nifti.nii")
        aff = nifti_img.affine
        if aff[0][1] != 0 or aff[0][2] != 0 or aff[1][0] != 0 or aff[1][2] != 0 or aff[2][0] != 0 or aff[2][1] != 0:
            rotated_cts.append(i)

In [ ]:
rotated_cts

In [ ]:
for rot in rotated_cts:
    if rot not in excluded:
        excluded.append(rot)

In [ ]:
len(excluded)

In [ ]:
print(excluded)

In [ ]:
with open("excluded_cts.pkl","wb") as f:
    pickle.dump(excluded,f)

In [ ]:
x_spacing[651]

In [ ]:
with open("processing/excluded_cts.pkl","rb") as f:
    excluded_cts = pickle.load(f)

In [ ]:
x_spacing_ex = []
y_spacing_ex = []
z_spacing_ex = []
for i in range(1,1251):
    if os.path.exists(f"processing/{i}/shoulder_nifti.nii") and (i not in excluded_cts):
        nifti_img = nib.load(f"processing/{i}/shoulder_nifti.nii")
        sx, sy, sz = nifti_img.header.get_zooms()
        x_spacing_ex.append(sx)
        y_spacing_ex.append(sy)
        z_spacing_ex.append(sz)
    else:
        x_spacing_ex.append([])
        y_spacing_ex.append([])
        z_spacing_ex.append([]) 

In [ ]:
min(x_spacing_ex),max(x_spacing_ex),min(y_spacing_ex),max(y_spacing_ex),min(z_spacing_ex),max(z_spacing_ex)

In [ ]:
len(x_spacing_ex)